In [1]:
import os

import numpy as np
import pandas as pd
import json

from glob import glob
from tqdm import tqdm

from torchvision.io import read_image
from torchvision.ops import masks_to_boxes
import scipy.io
import matplotlib.pyplot as plt

Data from the Public Dataset OASBUD contain images and semantic segmentation masks. 

See file "colab_Oasbud_convert" for details of how to download OASBUD images and masks.

This script is to convert all the semantic segmentation masks to .json files in Labelme format (same as data provided by Sponsor)

# Get Labels

"label_list" contains ground truth labels which is achieved from the file "colab_Oasbud_convert".

In [31]:
# Use Label list from Colab TFDS Oasbud Addition Demo
label_list = [0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0]


# Generate Jsons

## Update Images & Masks Names

In [6]:
image_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/OASBUD/imgs/', "*.png"))
mask_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/OASBUD/masks/', "*.png"))

print(f"Total Image Numbers: {len(image_list)}")
assert len(image_list) == len(mask_list)

In [69]:
# for item in image_list:
for item in mask_list:
    current_file_name = item
    new_file_name = os.path.dirname(item) + '/' + os.path.basename(item)[5:]
    os.rename(current_file_name, new_file_name)

## Genrate jsons using masks

In [33]:
image_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/OASBUD/imgs/', "*.png"))
mask_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/OASBUD/masks/', "*.png"))
image_list.sort()
mask_list.sort()

In [38]:
for i, mask_path in tqdm(enumerate(mask_list)):
    mask = read_image(mask_path)
    width, height = mask.shape[2], mask.shape[1]
    
    bool_mask = mask > 0
    box = masks_to_boxes(bool_mask)

    img_path = os.path.basename(mask_path).split('_mask')[0] + '.png'

    json_path = os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/OASBUD/jsons/', img_path.split('.')[0] + '.json')

    image_id = int(os.path.basename(mask_path).split('_')[1])
    label_id = label_list[image_id]

    if label_id == 0:
        label = 'benign'
    elif label_id == 1:
        label = 'malignant'

    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            label_dic = json.load(f)
            
        shapes = {"label" : label,
                    "points" : [
                            [float(box[0][0]), float(box[0][1])],
                            [float(box[0][2]), float(box[0][3])]
                    ],
                    "group_id": None,
                    "shape_type": "rectangle",
                    "flags": {}
                    }
        label_dic['shapes'].append(shapes)
        
    else:
        label_dic = {"version" : "5.0.1",
             "flags" : {},
             "shapes" : [],
             "imagePath" : img_path,
             "imageData": None,
             "imageHeight" : height,
             "imageWidth" : width
             }
        
        shapes = {"label" : label,
            "points" : [
                    [float(box[0][0]), float(box[0][1])],
                    [float(box[0][2]), float(box[0][3])]
            ],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
            }
        label_dic['shapes'].append(shapes)

    with open(json_path, 'w') as f:
        json.dump(label_dic, f)

200it [00:04, 47.36it/s]
